In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import itertools
import time

In [2]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

## Method to scrape the data(Comments) from the given source page

In [3]:
#Scrapping code
def Scrape(source, m1, m2, y1, y2):
    flag = 0
    month =  ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    if m2<m1:
        flag = 1
    soup = BeautifulSoup(source,'html.parser')
    dates = soup.findAll('span' , attrs={'class':'a-size-base a-color-secondary review-date'})
    text = soup.findAll('span', attrs={'class':'a-size-base review-text review-text-content'})
    
    #attrs={'class':'js-event-list-tournament-events'}
    # class_="a-size-base a-color-secondary review-date"  class_="a-size-base review-text review-text-content"
    
    d = []
    for (date,element) in  zip(dates,text):
        temp = date.text.lower().split(" ")
        yr1 = temp[-1]
        yr = int(yr1)
        mon1 = temp[-2]
        if any (x in mon1 for x in month):
            mon1 = temp[-2]
        else:
            mon1 = temp[-3]
        mon = month.index(mon1)
        if((mon>=m1 and mon<=m2)and(yr>=y1 and yr<=y2) and flag==0)or(flag==1 and ((mon<=11 and mon>=m1)or(mon<=m2 and mon>=1))):
            t = element.text.strip()
            if "    "in t:
                t = t.split("    ")[1]
                t = t.split("\n")[0]
            else:
                t = t.split("\n")[0]
            d.append(t)
    return d

## Opening the web driver and accessing the Robocraze amazon page

In [4]:
#opening web driver
loc = "D:\CAP\chromedriver\chromedriver.exe"
driver = webdriver.Chrome(loc)

home_link = "https://www.amazon.in/stores/roboCraze/roboCraze/page/19235B68-0603-400B-8CDE-71F79A088722"
driver.get("https://www.amazon.in/stores/roboCraze/roboCraze/page/19235B68-0603-400B-8CDE-71F79A088722")

driver.maximize_window()
time.sleep(2)

C:\Users\ZAID\AppData\Local\Temp\ipykernel_48000\237669999.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(loc)


### If required by the user to enter the range of month and year to search comments

In [5]:
#setting range to scrape reviews from

from_month , from_year  = input("Input from Month and year separated by a space").lower().split(" ")
to_month , to_year = input("Input to Month and year separated by a space").lower().split(" ")

month =  ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
m1 = month.index(from_month)
m2 = month.index(to_month)
y1 = int(from_year)
y2 = int(to_year)
if y2>y1:
    if m1<=m2:
        m1=0
        m2=11

Input from Month and year separated by a spaceJanuary 2019
Input to Month and year separated by a spaceApril 2019


## Selenium is used here to navigate into each category -> product -> comments

In [6]:
#count of number of element per page

lnks = driver.find_elements(By.CLASS_NAME, 'EditorialTile__innerContent__n92i8')
len(lnks)
# lnks[3].click()

14

In [7]:
#code to navigate through each item and their comment till last page
 
data = []

for i in range(1,12): 
# 1-13 as the website has 14 clickable items with 2 of it not being any category

    time.sleep(3)
    lnks = driver.find_elements(By.CLASS_NAME, 'EditorialTile__innerContent__n92i8')
    #EditorialTile__innerContent__3PswW
    lnks.pop(11)
    head = lnks[i].text
    print(head)
    lnks[i].click()
    temp_url = driver.current_url
    lnks1 = driver.find_elements(By.XPATH, '//a[@class="Overlay__overlay__LloCU ProductGridItem__overlay__IQ3Kw"]')
    x = len(lnks1)
    print(x)
    flag = 1
    for j in range(0,x):
        lnks1 = driver.find_elements(By.XPATH, '//a[@class="Overlay__overlay__LloCU ProductGridItem__overlay__IQ3Kw"]')
        lnks1[j].click()
        try:
            link2 = driver.find_element(By.XPATH, '//a[@class="a-link-emphasis a-text-bold"]')
            link2.click()
            data = data + (Scrape(driver.page_source, m1, m2, y1, y2))
            c = 0
            while True:
                try:
                    time.sleep(2)
#                     driver.navigate().refresh()
                    if driver.find_element(By.XPATH, '//li[@class="a-last"]'):
                        nxt_pg = driver.find_element(By.XPATH, '//li[@class="a-last"]')
                        nxt_pg.click()
                        data = data + (Scrape(driver.page_source, m1, m2, y1, y2))
                except NoSuchElementException:
                    driver.get(temp_url)
                    flag = 0
                    break
        except NoSuchElementException:
            driver.get(temp_url)
            time.sleep(2)
            flag = 0
        flag = 1
    driver.get(home_link)

QUADCOPTER
24


KeyboardInterrupt: 

In [8]:
data

['Defective!! Firstly, Got a used  product, starting from the box to the way the remote looks, it has been used. Thr paints come off at many places. Secondly, it does not power on when new batteries are put. Do not recommend seller',
 'Receiver of transmitter not  working no led indication, defective item received. What is the use of diffective item.',
 'CD IS A BROKEN ONE',
 'Good products']

In [9]:
len(data)

4

### Converting the Scraped data list into a dataframe and storing in an excel

In [10]:
df = pd.DataFrame()
df["Comments"] = data
df.head

<bound method NDFrame.head of                                             Comments
0  I buy this but in pakage  binding cable is not...
1  I buy this but in pakage  binding cable is not...
2  Defective!! Firstly, Got a used  product, star...
3                                 CD IS A BROKEN ONE>

In [11]:
df.to_excel("Comments.xlsx")
df

,Comments
0,I buy this but in pakage binding cable is not...
1,I buy this but in pakage binding cable is not...
2,"Defective!! Firstly, Got a used product, star..."
3,CD IS A BROKEN ONE


In [ ]:
df.count_values()
#ignore the comments fir the beter
